# *Escapa, con la llave*. 
## Organización del código, en *clases*.



**Este es el primer *Notebook* asociado a la primera práctica obligatoria**.


***TENED EN CUENTA** que habrá un segundo Notebook en el que se consigue que juegue solo usando A**

----------------------------------------

En esta primera parte de la práctica se va a estructurar el código de las funciones básicas (*Notebook* anterior) en clases, para así poder dejar el código listo para su adaptación al modo automático, usando *A\** (cosa que se hará en las siguientes sesiones).

------------------



Para este *Notebook*, el profesor proporciona un conjunto de clases que el alumno no tiene que modificar, pero puede hacerlo si lo desea.

El profesor proporciona:
1. Este *Notebook*, con las explicaciones y el ***main*** 
2. La *clase* **ui**, la cual implementará los controles (botones, desplegables, y la representación visual del juego)
3. La *clase* **Mediator** que pone en contacto los componentes gráficos entre sí, y utiliza los métodos de la clase **State**, **Level**, **Model**, y la clase **Loader**
4. **State**: Funciones para la construcción y consulta de estados (jugadores, cajas ...) <--> Modulo **JEscapa** (**JEscapa.py**).
5. **Level**: Funciones para la construcción y consulta de niveles (tablero, destinos) <--> Modulo **JEscapa** (**JEscapa.py**).

## Descripción de las clases

La representación del estado y de los niveles visto en el notebook de funciones básicas se ha organizado en clases.

La clase **State**: 
- tiene un constructor que recibe el jugador, la(s) piedra(s) y la(s) casilla(s) de agua. 
- tiene un método **get_piedras** que devuelve las piedras
- tiene un método **get_agua** que devuelve las coordenadas con agua

- tiene un método **\_\_str\_\_** que permite obtener la representación en modo texto del objeto y un método **\_\_repr\_\_** equivalente, para usarlo dentro de colecciones.

- Tiene también un método que permite determinar si dos estados son iguales y también otro para calcular su *hash*.

La clase **Level**:
- tiene un constructor que recibe el tablero y los destinos
- tiene un método **get_tablero** que devuelve el tablero, y las paredes.
- tiene un método **get_destino** que devuelve el destino al que se debe dirigir el jugador
- tiene un método **\_\_str\_\_** que permite obtener la representación en modo texto del objeto, así como un método **\_\_repr\_\_** equivalente

**Loader**, **UI**, **Mediator**



In [15]:
# Cargar Celda
# Ejemplo de visualización de un nivel
from typing import Tuple, List, Set
from UIJEscapaVPedro import gui
from ipywidgets import HTML


El alumno debe completar la implementación de los métodos de este notebook:


## Funciones a implementar por el alumno.

Nota: En el notebook anterior se usaban variables globales. En este notebook los valores están encapsulados en objetos. Pero la funcionalidad es la misma.

- move(level,state,mov): Recibe un nivel, un estado y un movimiento. **Devuelve un nuevo estado** resultante de aplicar el movimiento. Esta función puede reutilizar las funciones del *Notebook* anterior.

- is_goal(state): Recibe un estado. Devuelve True si los dos jugadores son None (los dos jugadores han llegado a la meta y es un estado final)

**NOTA - IMPORTANTE**: No debes borrar **# Cargar Celda** en cada una de las celdas en que aparezca.

In [16]:
# Cargar Celda
# No tocar esta primera linea

# Completa las funciones de esta celda y pruebalas en celdas posteriores
from EscapaConBooleanoLlave import Level, State

def is_valid(tablero: Tuple[Tuple[int, ...]],
             coord: Tuple[int, int]
             ) -> bool:
    """ Comprueba si una celda es váida y si es posible poner una piedra en esas coordenadas
    
    
    Parámetros
    ----------
    coord -- coordenadas de la casilla a analizar
    
    Devolución
    --------
    Devuelve True si la casilla es válida y no existe piedra ni cualquier jugador any player
    """  
    y,x = coord
    alto,ancho = len(tablero),len(tablero[0])
    return 0 <= y < alto and 0 <= x < ancho and tablero[coord[0]][coord[1]] ==1

def is_piedra_free(tablero: Tuple[Tuple[int, ...]],
                   piedra: Set[Tuple[int, int]],
                   coord: Tuple[int, int]
                   ) -> bool:
    """ Comprueba si una casilla es válida y si es posible poner el jugador en esas coordenadas
    
    
    Parámetros
    ----------
    coord -- coordenadas de la casilla a analizar
    
    Devolución
    --------
    Devuelve True si la casilla es válida y no existe piedra o agua.
    """   
    
    return is_valid(tablero, coord) and not tuple(coord) in piedra


def is_player_free(tablero: Tuple[Tuple[int, ...]],
                   enemigo: Tuple[int,int],
                   aspas: Set[Tuple[int, int]],
                   piedra: Set[Tuple[int, int]],
                   agua: Set[Tuple[int,int]],
                   coord: Tuple[int, int]
                   )-> bool:
    return is_piedra_free(tablero, piedra, coord) and not (coord in agua or  coord in aspas or coord in enemigo)

In [17]:
# Cargar Celda
def move_player(tablero: Tuple[Tuple[int, ...]],
                player: List[int],
                enemigo: Tuple[int, int],
                piedra: Set[Tuple[int, int]],
                agua: Set[Tuple[int,int]],
                aspas: Set[Tuple[int,int]],
                llave: Tuple[int, int],
                destino: Tuple[int, int],
                tiene_llave: bool,
                mov: Tuple[int, int]
                ) -> State:
    
    nueva_llave = set(llave)
    
    nuevoJ = tuple(map(lambda x,y: x+y,player,mov))
        
    if not is_player_free(tablero, enemigo, aspas, piedra, agua, nuevoJ):
        return State(player,piedra,agua,aspas,llave, tiene_llave)
    
    
    if nuevoJ == destino:
        if tiene_llave:
            return State(None, piedra, agua, aspas, nueva_llave, tiene_llave)
        else:
            # Si no tiene llave, no se puede avanzar:
            return State(player,piedra,agua,aspas ,llave, tiene_llave)
    elif nuevoJ in llave and not tiene_llave:
        nueva_llave.remove(tuple(nuevoJ))
        return State(list(nuevoJ), piedra, agua, aspas, nueva_llave, True)

    # Es una casilla cualquiera
    return State(list(nuevoJ), piedra, agua, aspas, llave, tiene_llave)
    
    

In [18]:
# Cargar Celda
def move_piedra(tablero: Tuple[Tuple[int, ...]],
                player: List[int],
                enemigo: Tuple[int,int],
                piedras: Set[Tuple[int,int]],
                aguas: Set[Tuple[int,int]],
                aspas: Set[Tuple[int,int]],
                llaves: Tuple[int, int],
                tiene_llave: bool,
                mov: Tuple[int, int]
                ) -> State:

    jugador_actual = player    
        
    nueva_piedra = set(piedras)
    nueva_agua = set(aguas)
    nueva_llave = set(llaves)
    nueva_aspa = set(aspas)
        
    nuevaCoord = list(map(lambda x,y: x+y,jugador_actual,mov))
    nuevaCoord2 = list(map(lambda x,y: x+y,nuevaCoord,mov))
    
    
    if tuple(nuevaCoord) in piedras and is_piedra_free(tablero, piedras, nuevaCoord2):
        nueva_piedra.remove(tuple(nuevaCoord))
        
        if tuple(nuevaCoord2) in aguas:
            nueva_agua.remove(tuple(nuevaCoord2))
        
        
        elif tuple(nuevaCoord2) in aspas:
            nueva_aspa.remove(tuple(nuevaCoord2))
            nueva_piedra.add(tuple(nuevaCoord2))
            
            
            y,x = nuevaCoord2
            alto,ancho = len(tablero),len(tablero[0])
                        
            
            for i in range(y+1,alto-1,1):
                            
                nuevaCoord3 = i,x
                nueva_aspa.remove(tuple(nuevaCoord3))
                                
                
        else:
            nueva_piedra.add(tuple(nuevaCoord2))               
    
    
    return State(player,nueva_piedra,nueva_agua,nueva_aspa,llaves, False)

In [19]:
# Cargar Celda
def move(level: Level, state: State, mov: Tuple[int, int]):
    """ Mueve al jugador: [1,0] (Arriba), [-1,0] (Abajo), [0,1] (Derecha), [0,-1] (Izquierda)
    
    Parámetros
    ----------
    level -- Un "nivel"
    State -- Un "estado"
    mov -- Dirección del movimiento
    
    Devolución
    --------
    Devuelve un nuevo estado, o el mismo, si el movimiento indicado no es posible.
    """
        
    piedra = state.get_piedra()
    agua = state.get_agua()
    aspa=state.get_aspa()
    llave = state.get_llave()
    tiene_llave = state.tiene_llave()
    player = state.get_player()
    movimiento = mov
        
    tablero = level.get_tablero()
    destino = level.get_destino()
    
    enemigo = level.get_enemigo()
        
    e1= move_piedra(tablero, player, enemigo, piedra, agua, aspa, llave, tiene_llave, mov)
    e2 = move_player(tablero, player, enemigo, e1.get_piedra(), e1.get_agua(), e1.get_aspa(), llave,destino, tiene_llave, mov)
    
    return e2

In [20]:
# Cargar Celda
def is_goal(state):
    """ Determinación de si "es el objetivo" 
    
    Parámetros
    ----------
    state -- Un "estado"
    
    Devolución
    --------
    Devuelve "True" si el "estado" es un "objetivo"
    """
    player = state.get_player()
    return player is None



Una vez hayas implementado y probado las funciones, trata de ejecutar el juego.

El **Mediador** es la clase que sirve de enganche entre las clases de interfaz y la funcionalidad. Leerá las celdas que empiezan por "# Cargar Celda", cargará esas funciones dinámicamente, y las usará para mover y comprobar si la partida finaliza.


# Atención
Es código de abajo carga el *Notebook* actual, por lo que si queremos que el funcionamiento se actualize debemos guardar el *Notebook* y recargar todo el proyecto.

In [21]:
from IPython.display import display
from MediadorVPedroConBooleanoLlave import Mediator
from UIJEscapaVPedro import gui
import warnings
warnings.filterwarnings("ignore")
ui = gui(manual = True)

med = Mediator.get_instance(modelPath = "JEscapaManualConBooleanoLlave.ipynb")
med.register_ui(ui)


# Cuando llamo a la función se crean
display(ui.get_ui_elements())


